# Este es un ejemplo de un modelo de CRFs + features para resolver la tarea de NER, usando la herramienta crfsuite  referenciado en la página https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb. El rendimiento del clasificador es del 80%. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

## Uso de CoNLL2002 para  reconocimiento de entidades +CRF

El corpis de CoNLL2002 tiene especificados los archivos de los conjuntos de entrenamiento, evaluación y testeo. 

In [3]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [4]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
print(test_sents[4])

[('Los', 'DA', 'O'), ('clientes', 'NC', 'O'), ('pueden', 'VMI', 'O'), ('utilizar', 'VMN', 'O'), ('el', 'DA', 'O'), ('portal', 'NC', 'O'), ('"', 'Fe', 'O'), ('viajesydestinos.com', 'NC', 'B-MISC'), ('"', 'Fe', 'O'), (',', 'Fc', 'O'), ('que', 'PR', 'O'), ('el', 'DA', 'O'), ('pasado', 'AQ', 'O'), ('año', 'NC', 'O'), ('recibió', 'VMI', 'O'), ('ya', 'RG', 'O'), ('más', 'RG', 'O'), ('de', 'SP', 'O'), ('medio', 'DN', 'O'), ('millón', 'NC', 'O'), ('de', 'SP', 'O'), ('visitas', 'NC', 'O'), ('de', 'SP', 'O'), ('Internautas', 'NC', 'B-MISC'), (',', 'Fc', 'O'), ('para', 'SP', 'O'), ('consultar', 'VMN', 'O'), ('tarifas', 'NC', 'O'), ('de', 'SP', 'O'), ('billetes', 'NC', 'O'), ('de', 'SP', 'O'), ('transporte', 'NC', 'O'), (',', 'Fc', 'O'), ('plazas', 'NC', 'O'), ('hoteleras', 'AQ', 'O'), ('o', 'CC', 'O'), ('paquetes', 'NC', 'O'), ('turísticos', 'AQ', 'O'), (',', 'Fc', 'O'), ('aunque', 'CS', 'O'), ('la', 'DA', 'O'), ('venta', 'NC', 'O'), ('final', 'AQ', 'O'), ('corre', 'VMI', 'O'), ('siempre', 'RG', 

In [5]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

## Selección de características

El modelo base sólo tiene como característica el token de la palabra.

sklearn-crfsuite y python-crfsuite soporta varios formatos de características.

In [6]:

 def shape(self,word):
        shape = ""
        for letter in word:
            if letter.isdigit():
                shape = shape + "d"
            elif letter.isalpha():
                if letter.isupper():
                    shape = shape + "W"
                else:
                    shape = shape + "w"
            else:
                shape = shape + letter
        return shape

# Características o features de forma y vecindad

1. Postag de  cada palabra con ventana a la izquierda y a la derecha
2. Palabras sólo minúsculas
3. Ventanas de palabras del token 
4. La palabra es título, es mayúscula, es dígito.
5. Postag de las palabras alrededor.

In [7]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features




def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [8]:
sent2features(train_sents[0])[0]

{'bias': 1.0,
 'word.lower()': 'melbourne',
 'word[-3:]': 'rne',
 'word[-2:]': 'ne',
 'word.isupper()': False,
 'word.istitle()': True,
 'word.isdigit()': False,
 'postag': 'NP',
 'postag[:2]': 'NP',
 'BOS': True,
 '+1:word.lower()': '(',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'Fpa',
 '+1:postag[:2]': 'Fp'}

Se extraen las características para cada sentencia de entrenamiento, sent2features; extrae las características de forma de cada palabra, sent2labels, obtiene la etiqueta de la palabra y sent2tokens el token de la palabra.

In [9]:
#print(sent2features(train_sents[0])[0])
print(sent2labels(train_sents[0])[0])
print(sent2tokens(train_sents[0])[0])

B-LOC
Melbourne


Se genera el conjunto de entrenamiento y el de testeo con la información de las características de forma del token y el postag.

In [10]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 469 ms, sys: 36.4 ms, total: 505 ms
Wall time: 504 ms


In [11]:
print(X_test[2])

[{'bias': 1.0, 'word.lower()': 'las', 'word[-3:]': 'Las', 'word[-2:]': 'as', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'DA', 'postag[:2]': 'DA', 'BOS': True, '+1:word.lower()': 'reservas', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'NC', '+1:postag[:2]': 'NC'}, {'bias': 1.0, 'word.lower()': 'reservas', 'word[-3:]': 'vas', 'word[-2:]': 'as', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'NC', 'postag[:2]': 'NC', '-1:word.lower()': 'las', '-1:word.istitle()': True, '-1:word.isupper()': False, '-1:postag': 'DA', '-1:postag[:2]': 'DA', '+1:word.lower()': '"', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'Fe', '+1:postag[:2]': 'Fe'}, {'bias': 1.0, 'word.lower()': '"', 'word[-3:]': '"', 'word[-2:]': '"', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'Fe', 'postag[:2]': 'Fe', '-1:word.lower()': 'reservas', '-1:word.istitle()'

## Entrenamiento del modelo usando CRFsuite
El algoritmo de entrenamiento está basado en el algoritmo L-BFGS con estándares de regularización. El algoritmo LBFGS está basado en el método de Newton Rapshon y sirve para optimización nolineal. Los CRFs son funciones de regresión logística nolineal.

In [12]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=200, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 43.6 s, sys: 16.8 ms, total: 43.7 s
Wall time: 43.7 s


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=200)

## Evaluación
Se evalua el desempeño del clasificador basado en CRF comparando la predicción del conjunto de testro X_test en contra del conjunto de testeo de etiquetas y_test.

In [13]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-LOC', 'B-ORG', 'B-PER', 'I-PER', 'B-MISC', 'I-ORG', 'I-LOC', 'I-MISC']

In [23]:
y_pred = crf.predict(X_test)
print(X_test[0])
print(y_pred[0])
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

[{'bias': 1.0, 'word.lower()': 'la', 'word[-3:]': 'La', 'word[-2:]': 'La', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'DA', 'postag[:2]': 'DA', 'BOS': True, '+1:word.lower()': 'coruña', '+1:word.istitle()': True, '+1:word.isupper()': False, '+1:postag': 'NC', '+1:postag[:2]': 'NC'}, {'bias': 1.0, 'word.lower()': 'coruña', 'word[-3:]': 'uña', 'word[-2:]': 'ña', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'NC', 'postag[:2]': 'NC', '-1:word.lower()': 'la', '-1:word.istitle()': True, '-1:word.isupper()': False, '-1:postag': 'DA', '-1:postag[:2]': 'DA', '+1:word.lower()': ',', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'Fc', '+1:postag[:2]': 'Fc'}, {'bias': 1.0, 'word.lower()': ',', 'word[-3:]': ',', 'word[-2:]': ',', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'Fc', 'postag[:2]': 'Fc', '-1:word.lower()': 'coruña', '-1:word.istitle()': True, '-1

0.8001188546978065

Inspect per-class results in more detail:

In [15]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

       B-LOC      0.813     0.787     0.800      1084
       I-LOC      0.697     0.643     0.669       325
      B-MISC      0.717     0.560     0.629       339
      I-MISC      0.725     0.621     0.669       557
       B-ORG      0.813     0.834     0.823      1400
       I-ORG      0.860     0.789     0.823      1104
       B-PER      0.844     0.888     0.865       735
       I-PER      0.878     0.940     0.908       634

   micro avg      0.815     0.791     0.803      6178
   macro avg      0.793     0.758     0.773      6178
weighted avg      0.812     0.791     0.800      6178



# Testeo de una sentencia que no está en el conjunto de testeo de conll2002
La sentencia de entrada tiene como entradas el token  y el postag ye estas deben ser preprocesadas con toda la información de las características de ventanas, forma, cercanía, etc. El clasificador deberá predecir la etiqueta correspondiente. Por lo tanto se debe realizar un preprocesamiento a cada palabra de la sentencia usando la función word2features con el fin de extraer el cunjunto de features de forma de la palabra y el postag.

In [34]:


prueba=[('La', 'DA'), ('Coruña', 'NC'), ('sería','VSI'), ('el','DA'), ('nuevo','AQ'), ('equipo','NC'), ('de','SP'), ('James','NP'),('Rodriguez','NP'),(',','Fc'),('aunque','CC'),('todavía','RG'),
        ('es','VSI'), ('de','SP'),('el','DA'), ('Real','NP'), ('Madrid','NP'), ('de','SP'), ('España','NP')]
prueba1= [('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'),
 ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


def pos_tag(sentence):
    sentence_features = [word2features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
#print(sentence_features) 
print(pos_tag(prueba))  # [('I', 'PRP'), ('am', 'VBP'), ('Bob', 'NNP'), ('!', '.')]

[(('La', 'DA'), 'B-LOC'), (('Coruña', 'NC'), 'I-LOC'), (('sería', 'VSI'), 'O'), (('el', 'DA'), 'O'), (('nuevo', 'AQ'), 'O'), (('equipo', 'NC'), 'O'), (('de', 'SP'), 'O'), (('James', 'NP'), 'B-PER'), (('Rodriguez', 'NP'), 'I-PER'), ((',', 'Fc'), 'O'), (('aunque', 'CC'), 'O'), (('todavía', 'RG'), 'O'), (('es', 'VSI'), 'O'), (('de', 'SP'), 'O'), (('el', 'DA'), 'O'), (('Real', 'NP'), 'B-ORG'), (('Madrid', 'NP'), 'I-ORG'), (('de', 'SP'), 'I-ORG'), (('España', 'NP'), 'I-ORG')]
